## Extracting bare soil reflectance data from known bare soil locations

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from matplotlib.ticker import MaxNLocator
import os

ee.Initialize()
ee.Initialize(project='ee-bd167')

In [ ]:
# Cloud masking function
def filterS2_level2A(image):
    SCL = image.select('SCL')
    mask01 = ee.Image(0).where(SCL.lt(8).And(SCL.gt(3)), 1)  # Put a 1 on good pixels
    return image.updateMask(mask01)

### Time series

In [ ]:
roi = ee.FeatureCollection('projects/ee-bd167/assets/Sampling_Fields')

s2_ROI = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2020-01-01', '2024-12-31') \
    .filterBounds(roi) \
    .sort('system:time_start', True) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20) \
    .map(filterS2_level2A);

# clip to roi
s2_ROI_clipped = s2_ROI.map(lambda image: image.clip(roi))

# select B1-B12 bands
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
s2_ROI_clipped = s2_ROI_clipped.select(bands)

# Function to convert DN to reflectance for a single image
def convert_dn_to_reflectance(image):
    # Define the scaling factors and offsets for Sentinel-2 bands (replace with actual values)
    scaling_factors = [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]
    offsets = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # Convert DN to reflectance for each band
    def dn_to_reflectance(band, scale, offset):
        return band.multiply(scale).add(offset)

    # Apply the conversion to the selected bands
    reflectance_image = image.multiply(scaling_factors).add(offsets)

    # Copy the properties from the original image to the new image
    reflectance_image = reflectance_image.copyProperties(image, image.propertyNames())

    return reflectance_image

s2_ROI_ref = s2_ROI_clipped.map(convert_dn_to_reflectance)


In [ ]:
s2_ROI_ref

In [ ]:
# Add spectral indices to the image collection

def add_indices(image):
    # Add NDVI
    image = image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))
    # Add NBR2
    image = image.addBands(image.normalizedDifference(['B12', 'B11']).rename('NBR2'))
    return image

s2_SI = s2_ROI_ref.map(add_indices)

# remove bands B1-B12
SIs = ['NDVI', 'NBR2']
s2_SI = s2_SI.select(SIs)

In [ ]:
# Function to get the mean spectral value for each field in the collection
def get_field_spectral_profile(image, field):
    # Reduce the image by mean over the field
    mean_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=field.geometry(),
        scale=10,  # Sentinel-2 resolution
        bestEffort=True
    )

    # Retrieve NDVI and NBR2 values from the dictionary, default to None if missing
    ndvi_value = mean_dict.get('NDVI', None)
    nbr2_value = mean_dict.get('NBR2', None)

    # Check if both NDVI and NBR2 have valid values (i.e., not None)
    valid_check = ee.Number(ee.Algorithms.If(ndvi_value, 1, 0)).multiply(ee.Number(ee.Algorithms.If(nbr2_value, 1, 0)))

    # Only proceed if we have valid data
    def set_field_properties():
        # Get the field's unique Name property (e.g., 'Buckie_3', 'Doncaster_1')
        field_name = field.get('Name')  # Update this to match the 'Name' column in your table

        # Extract the image date
        image_date = image.date().format('YYYY-MM-dd')

        # Create a dictionary with field name and spectral indices
        field_profile = mean_dict.set('image_date', image_date).set('field_name', field_name)
        return ee.Feature(None, field_profile)

    # Return feature with properties only if valid data exists, else return None
    return ee.Algorithms.If(valid_check, set_field_properties(), None)

# Function to process all images for a specific field and return a DataFrame
def process_field(field):
    # Apply the function to each image in the collection for this specific field
    SI_profiles = s2_SI.map(lambda image: get_field_spectral_profile(image, field), dropNulls=True)

    # Convert the results to a list of dictionaries for further use
    SI_profiles_list = SI_profiles.getInfo()

    # Convert the list to a pandas DataFrame with properties as columns
    if len(SI_profiles_list['features']) > 0:
        SI_profiles = pd.DataFrame([profile['properties'] for profile in SI_profiles_list['features']])

        # Convert 'image_date' to datetime format
        SI_profiles['image_date'] = pd.to_datetime(SI_profiles['image_date'], errors='coerce')

        # Separate NDVI and NBR2 into different rows
        SI_profiles_ndvi = SI_profiles[['field_name', 'NDVI', 'image_date']].copy()
        SI_profiles_ndvi['metric'] = 'NDVI'

        SI_profiles_nbr2 = SI_profiles[['field_name', 'NBR2', 'image_date']].copy()
        SI_profiles_nbr2['metric'] = 'NBR2'

        # Rename columns for consistency
        SI_profiles_ndvi = SI_profiles_ndvi.rename(columns={'NDVI': 'value'})
        SI_profiles_nbr2 = SI_profiles_nbr2.rename(columns={'NBR2': 'value'})

        # Combine NDVI and NBR2 into one DataFrame
        SI_profiles_combined = pd.concat([SI_profiles_ndvi, SI_profiles_nbr2])

        # Create a new row identifier 'fieldname_metric'
        SI_profiles_combined['fieldname_metric'] = SI_profiles_combined['field_name'] + '_' + SI_profiles_combined['metric']

        # Pivot the DataFrame to get timeseries with dates as columns
        pivoted_df = SI_profiles_combined.pivot_table(
            index='fieldname_metric',  # Each row will be 'fieldname_NDVI' or 'fieldname_NBR2'
            columns='image_date',      # Dates as columns
            values='value',            # NDVI or NBR2 values
            aggfunc='mean'             # In case there are duplicates
        )

        return pivoted_df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no valid data is available

# Process each field in the FeatureCollection and store DataFrames in a dictionary
field_dataframes = {}
fields = roi.getInfo()['features']

for field in fields:
    field_name = field['properties']['Name']  # Use 'Name' from the structure you provided
    field_dataframes[field_name] = process_field(ee.Feature(field))
# At this point, 'field_dataframes' contains a DataFrame for each field, identified by its 'Name'


In [ ]:
field_dataframes

In [ ]:
# Define a function to apply Savitzky-Golay smoothing to each row (field-metric pair)
def smooth_timeseries_sg(df, window_size, poly_order):
    """
    Apply Savitzky-Golay smoothing to all rows (field-metric pair) in the DataFrame.
    
    Args:
    - df: DataFrame containing the time series data (rows are field-metric pairs, columns are dates).
    - window_size: The size of the window (must be odd).
    - poly_order: The order of the polynomial to fit.
    
    Returns:
    - smoothed_df: DataFrame with the smoothed time series.
    """
    smoothed_df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original
    for idx, row in df.iterrows():  # Iterate through each row (field-metric pair)
        y = row.values  # Extract the values (time series)
        
        # Print original values for debugging
        print(f"Original {idx} values: {y}")
        
        # Check if the length of the series is greater than or equal to the window size
        if len(y) >= window_size and np.all(np.isfinite(y)):  # Ensure no NaN values in the data
            yhat = savgol_filter(y, window_size, poly_order)
            smoothed_df.loc[idx] = yhat  # Store the smoothed values
            
            # Print smoothed values for verification
            print(f"Smoothed {idx} values: {yhat}")
        else:
            print(f"Skipping smoothing for {idx} because length is less than the window size or contains NaN")
            smoothed_df.loc[idx] = y  # If not enough data or contains NaN, leave it unsmoothed
    return smoothed_df

# Apply the smoothing function to all DataFrames in field_dataframes
smoothed_field_dataframes = {}

for field_name, df in field_dataframes.items():
    # Apply the Savitzky-Golay smoothing function to each DataFrame
    print(f"\nSmoothing field: {field_name}")
    smoothed_df = smooth_timeseries_sg(df, window_size=9, poly_order=2)
    smoothed_field_dataframes[field_name] = smoothed_df  # Store in a new dictionary


In [ ]:
# Function to plot NDVI and NBR2 time series for a single field using dual y-axes
def plot_dual_axis_time_series(field_name, df):
    """
    Plot NDVI and NBR2 time series for a single field with dual y-axes.

    Args:
    - field_name: The name of the field (str).
    - df: DataFrame containing the time series data for NDVI and NBR2, with dates as columns.
    """
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Extract dates, NDVI, and NBR2 data
    dates = pd.to_datetime(df.columns)  # x-axis: Dates
    ndvi = df.loc[field_name + '_NDVI'].values  # y-axis 1: NDVI
    nbr2 = df.loc[field_name + '_NBR2'].values  # y-axis 2: NBR2

    # Plot NDVI on the first y-axis (ax1)
    ax1.set_xlabel('Date')
    ax1.set_ylabel('NDVI', color='tab:green')
    ax1.plot(dates, ndvi, color='tab:green', marker='.', label='NDVI')
    ax1.tick_params(axis='y', labelcolor='tab:green')

    # Create a second y-axis (ax2) that shares the same x-axis
    ax2 = ax1.twinx()
    ax2.set_ylabel('NBR2', color='tab:orange')
    ax2.plot(dates, nbr2, color='tab:orange', marker='.', label='NBR2')
    ax2.tick_params(axis='y', labelcolor='tab:orange')

    # Set the title
    plt.title(f'{field_name}')

    # Use MaxNLocator to set more x-ticks
    ax1.xaxis.set_major_locator(MaxNLocator(20))

    # # Add minor ticks to x-axis
    # ax1.xaxis.set_minor_locator(plt.MultipleLocator(20))

    # Format the x-axis to show dates cleanly
    fig.autofmt_xdate()

    # Show the plot
    plt.tight_layout()
    plt.show()

for field_name, df in smoothed_field_dataframes.items():
    plot_dual_axis_time_series(field_name, df)


In [ ]:
# Print the date with lowest NDVI before the peaks
for field_name, df in smoothed_field_dataframes.items():
    ndvi_values = df.loc[field_name + '_NDVI'].values
    ndvi_dates = pd.to_datetime(df.columns)
    ndvi_peaks = np.where(ndvi_values == np.max(ndvi_values))[0]
    ndvi_peak_date = ndvi_dates[ndvi_peaks[0]]
    print(f"Field: {field_name}, Peak NDVI: {np.max(ndvi_values)}, Peak Date: {ndvi_peak_date}")

    # Find the date with the lowest NDVI before the peak
    ndvi_values_before_peak = ndvi_values[:ndvi_peaks[0]]
    ndvi_dates_before_peak = ndvi_dates[:ndvi_peaks[0]]
    lowest_ndvi_before_peak = np.min(ndvi_values_before_peak)
    lowest_ndvi_date_before_peak = ndvi_dates_before_peak[np.where(ndvi_values_before_peak == lowest_ndvi_before_peak)[0][0]]
    print(f"Lowest NDVI before peak: {lowest_ndvi_before_peak}, Date: {lowest_ndvi_date_before_peak}")

In [ ]:
# Define the directory to save the CSV files
output_dir = "C:\\Users\\bd167\\OneDrive - University of Leicester\\Documents\\Data\\Jupyter\\Sampling\\Reflectance\\"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save each DataFrame to a CSV file
file_paths = {}
for field_name, df in smoothed_field_dataframes.items():
    file_path = os.path.join(output_dir, f"{field_name}_smoothed.csv")
    df.to_csv(file_path)
    file_paths[field_name] = file_path

# Provide the file paths of the saved CSV files to the user
file_paths